## Building A Chatbot
We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions


In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY") # type: ignore
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY") # type: ignore

In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama-3.1-8b-instant")
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x1552e9af0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1570b77a0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Luis and I am an AI Engineer")])

AIMessage(content="Nice to meet you, Luis. As an AI Engineer, I'm sure you're working on some exciting projects. What area of AI are you most interested in or currently working on? Is it NLP (Natural Language Processing), Computer Vision, Reinforcement Learning, or something else?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 47, 'total_tokens': 105, 'completion_time': 0.07138272, 'completion_tokens_details': None, 'prompt_time': 0.002336767, 'prompt_tokens_details': None, 'queue_time': 0.005314944, 'total_time': 0.073719487}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9ca2574dca', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--2ce2d3f4-9879-4a9a-aead-6ffeb8e01ef7-0', usage_metadata={'input_tokens': 47, 'output_tokens': 58, 'total_tokens': 105})

In [4]:
### Any Conversation inside a list, it will be able to remember the previous context

from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi, My name is Luis and I am an AI Engineer"),
        AIMessage(content="Nice to meet you, Luis. I'm happy to chat with you about AI engineering. What specific areas of AI engineering are you working on or interested in? Are you working with machine learning models, natural language processing, computer vision, or something else?"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="Your name is Luis and you're an AI Engineer.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 119, 'total_tokens': 131, 'completion_time': 0.014959004, 'completion_tokens_details': None, 'prompt_time': 0.00816176, 'prompt_tokens_details': None, 'queue_time': 0.005148995, 'total_time': 0.023120764}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9ca2574dca', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--4135e506-f74f-4fea-b464-5a102c5a2aab-0', usage_metadata={'input_tokens': 119, 'output_tokens': 12, 'total_tokens': 131})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_messages_history = RunnableWithMessageHistory(model, get_session_history)

In [6]:
config = {"configurable":{"session_id": "chat1"}}

In [7]:
response = with_messages_history.invoke(
    [HumanMessage(content="Hi, My name is Luis and I am an AI Engineer")],
    config=config
)

In [8]:
response.content

"Nice to meet you, Luis. You're an AI Engineer, that's really cool. What kind of projects are you currently working on or interested in?"

In [9]:
with_messages_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config
)

AIMessage(content="Your name is Luis, and you're an AI Engineer.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 93, 'total_tokens': 107, 'completion_time': 0.015403697, 'completion_tokens_details': None, 'prompt_time': 0.007154272, 'prompt_tokens_details': None, 'queue_time': 0.009659934, 'total_time': 0.022557969}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_6c980774ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--6bd15302-472c-4c5a-ae46-db62f1a02374-0', usage_metadata={'input_tokens': 93, 'output_tokens': 14, 'total_tokens': 107})

In [10]:
### Change the config ---> session_id
config1 = {"configurable":{"session_id": "chat2"}}
response = with_messages_history.invoke(
    [HumanMessage(content="What my name?")],
    config=config1
)
response.content

"I don't have any information about your name. I'm a large language model, I don't have the ability to retain personal information or recall previous conversations. Each time you interact with me, it's a new conversation. If you'd like to share your name, I'd be happy to chat with you."

In [11]:
response = with_messages_history.invoke(
    [HumanMessage(content="My name is Enrique")],
    config=config1
)
response.content

"Hola Enrique. It's nice to meet you, even if it's a virtual meeting. Is there something I can help you with or would you like to chat about something in particular?"

In [12]:
response = with_messages_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config1
)
response.content

'Your name is Enrique.'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the questions to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [14]:
chain.invoke({"messages": [HumanMessage(content="Hi my name is Luis")]})

AIMessage(content="Hello Luis, it's nice to meet you. I'm here to help with any questions or topics you'd like to discuss. How's your day going so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 56, 'total_tokens': 92, 'completion_time': 0.04040816, 'completion_tokens_details': None, 'prompt_time': 0.005585749, 'prompt_tokens_details': None, 'queue_time': 0.007131696, 'total_time': 0.045993909}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_6c980774ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--8de64a00-b675-4579-97a0-66edd8ca3673-0', usage_metadata={'input_tokens': 56, 'output_tokens': 36, 'total_tokens': 92})

In [15]:
with_messages_history = RunnableWithMessageHistory(chain, get_session_history)

In [16]:
config = {"configurable":{"session_id": "chat3"}}
response = with_messages_history.invoke(
    [HumanMessage(content="Hi my name is Luis")],
    config = config
    )
response

AIMessage(content="Hi Luis, it's nice to meet you. I'm here to help with any questions or topics you'd like to discuss. How's your day going so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 56, 'total_tokens': 92, 'completion_time': 0.03702006, 'completion_tokens_details': None, 'prompt_time': 0.004984662, 'prompt_tokens_details': None, 'queue_time': 0.009532965, 'total_time': 0.042004722}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_6c980774ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--f1bab1e1-65b0-474f-958c-296ac8bff73c-0', usage_metadata={'input_tokens': 56, 'output_tokens': 36, 'total_tokens': 92})

In [17]:
### Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the questions to the best of your ability in {language}. "),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [18]:
response = chain.invoke({"messages": [HumanMessage(content="Hi my name is Luis")], "language": "Spanish"})
response

AIMessage(content='Hola Luis, estoy aquí para ayudarte. ¿En qué puedo ayudarte hoy?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 59, 'total_tokens': 77, 'completion_time': 0.019867954, 'completion_tokens_details': None, 'prompt_time': 0.002953672, 'prompt_tokens_details': None, 'queue_time': 0.005641824, 'total_time': 0.022821626}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9ca2574dca', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--68d39bf6-27ea-4fff-80d3-85cb694988dc-0', usage_metadata={'input_tokens': 59, 'output_tokens': 18, 'total_tokens': 77})

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [19]:
with_messages_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [23]:
config = {"configurable":{"session_id": "chat4"}}
response = with_messages_history.invoke(
    {"messages": [HumanMessage(content="Hi, I am Luis Enrique")], "language": "Spanish"},
    config = config
)
response

AIMessage(content='¡Hola Luis Enrique! ¡Es un gusto conocerte! ¿Qué te trae hoy aquí? ¿Quieres hablar sobre algo en particular o simplemente charlar?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 189, 'total_tokens': 225, 'completion_time': 0.052353284, 'completion_tokens_details': None, 'prompt_time': 0.018345877, 'prompt_tokens_details': None, 'queue_time': 0.006502559, 'total_time': 0.070699161}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_6c980774ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--f66a8bc8-7755-4909-a752-de831b4e6dfa-0', usage_metadata={'input_tokens': 189, 'output_tokens': 36, 'total_tokens': 225})

In [ ]:
response = with_messages_history.invoke(
    {"messages": [HumanMessage(content="Wha it is my name")], "language": "Spanish"},
    config = config
)

response

AIMessage(content='Su nombre es Luis Enrique. ¿Necesitas recordar algo? ¿O simplemente querías mencionarlo de nuevo?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 239, 'total_tokens': 268, 'completion_time': 0.049272547, 'completion_tokens_details': None, 'prompt_time': 0.018936963, 'prompt_tokens_details': None, 'queue_time': 0.005429424, 'total_time': 0.06820951}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_6c980774ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--d0e9020a-13b5-4964-9214-46cbf1e416fc-0', usage_metadata={'input_tokens': 239, 'output_tokens': 29, 'total_tokens': 268})

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [42]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(
    max_tokens = 45,
    strategy="last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on="human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [43]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages = itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {"messages": messages + [HumanMessage(content = "What ice cream do I like?")],
     "language": "Spanish"}
)

response

AIMessage(content='Lo siento, no tengo esa información. ¿Quieres saber algunas opciones de helado que podrían gustarte?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 114, 'total_tokens': 139, 'completion_time': 0.04347068, 'completion_tokens_details': None, 'prompt_time': 0.00745076, 'prompt_tokens_details': None, 'queue_time': 0.005385393, 'total_time': 0.05092144}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9ca2574dca', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--add509e6-5e7c-4f59-90e8-e4892d4eba1a-0', usage_metadata={'input_tokens': 114, 'output_tokens': 25, 'total_tokens': 139})

In [35]:
response = chain.invoke(
    {"messages": messages + [HumanMessage(content = "What math problem did I ask?")],
     "language": "Spanish"}
)

response

AIMessage(content='Tú me preguntaste una cuestión de matemáticas básica: 2 + 2.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 114, 'total_tokens': 139, 'completion_time': 0.03522799, 'completion_tokens_details': None, 'prompt_time': 0.010997864, 'prompt_tokens_details': None, 'queue_time': 0.012881578, 'total_time': 0.046225854}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_6c980774ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--1b223907-1e11-4154-a476-51fe5e16524c-0', usage_metadata={'input_tokens': 114, 'output_tokens': 25, 'total_tokens': 139})

In [36]:
### Lets wrap this in the Message History
with_messages_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

config = {"configurable": {"session_id": "chat5"}}

In [38]:
response = with_messages_history.invoke(
    {"messages": [HumanMessage(content="Wha it is my name")], "language": "Spanish"},
    config = config
)

response

AIMessage(content='Lo siento, pero no sé tu nombre. ¿Puedes decirme cómo te llamas? Estoy aquí para ayudarte y responder a tus preguntas.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 60, 'total_tokens': 96, 'completion_time': 0.045280444, 'completion_tokens_details': None, 'prompt_time': 0.005001554, 'prompt_tokens_details': None, 'queue_time': 0.012321839, 'total_time': 0.050281998}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_6c980774ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--74880f26-bd22-49c2-af1b-604f14720821-0', usage_metadata={'input_tokens': 60, 'output_tokens': 36, 'total_tokens': 96})

In [39]:
response = with_messages_history.invoke(
    {"messages": [HumanMessage(content="What math problem did I ask?")], "language": "Spanish"},
    config = config
)

response

AIMessage(content='Lo siento, pero no me parece que hayas hecho ninguna pregunta matemática hasta ahora. ¿Puedes repetir tu pregunta para que pueda ayudarte?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 61, 'total_tokens': 96, 'completion_time': 0.068892672, 'completion_tokens_details': None, 'prompt_time': 0.003546266, 'prompt_tokens_details': None, 'queue_time': 0.005581016, 'total_time': 0.072438938}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9ca2574dca', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--f2618209-2ec3-4b4a-8b3a-c54bc20357cf-0', usage_metadata={'input_tokens': 61, 'output_tokens': 35, 'total_tokens': 96})